In [ ]:
import json
import os
from collections import defaultdict
from IPython.display import HTML
import numpy as np
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

from analysis import (
    parse_results, get_token_groups, plot_token_groups, animate, average_length_by_weight,
    tokens, counter_tokens, total_of, sum_up, mk_cumulative_interpolator
)
%matplotlib inline

In [ ]:
def read(path):
    tout, Cout, info = parse_results(path)
    tokg, ctrg, spg = get_token_groups(Cout)
    init = []
    for sk in spg['polymeric']:
        if Cout[sk][0] > 0:
            init.append((sk, Cout[sk][0]))
    (sk0, ic0), = init
    return locals()

In [ ]:
TOKEN = 'Pnointernoloopscaling'
cases = __import__('cases_%s' % TOKEN)
res = lambda **kw: ('%s-res/%s_{mf}_{tp}.txt' % (TOKEN, TOKEN)).format(**kw)

In [ ]:
rs = dict()
for mf in range(len(cases.mass_fracs)):
    for tp in range(len(cases.pulse_durs)):
        rs[mf, tp] = read(res(mf=mf, tp=tp))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
for i, k in enumerate(rs[0, 0]['spg']['ord_rad']):
    ax.loglog(rs[0, 0]['tout'], rs[0, 0]['Cout'][k], ls=['-',':','-.'][i % 3], label=k)
ax.set_ylim([1e-15, 100])
ax.legend()
print(rs[len(cases.mass_fracs)-1, 0]['ic0'])

In [ ]:
nmf, npd = len(cases.mass_fracs), len(cases.pulse_durs)
fig, axes = plt.subplots(nmf, npd, figsize=(14, 14))
for mf in range(len(cases.mass_fracs)):
    for tp in range(len(cases.pulse_durs)):
        ax, r = axes[mf, tp], rs[mf, tp]
        tot = total_of(r['Cout'], r['tokg']['a'])
        ax.plot(r['tout'], tot)#/r['ic0'])
        np.savetxt('%s_pmf%d_pt%d.txt' % (
            TOKEN, 
            int(-np.log10(cases.mass_fracs[mf])),
            int(-np.log10(cases.pulse_durs[tp]))
        ), np.array([r['tout'], tot]).T)
        ax.vlines([cases.pulse_durs[tp]], 0, 1, transform=ax.get_xaxis_transform(), lw=0.5, linestyle='--')
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlim([1e-9, 1e2])
        #ax.set_ylim([1e-1, 1e2])
        ax.set_ylim([1e-8, 1e-4])
        if tp == 2:
            ax.set_ylabel('mass frac = %s' % str(cases.mass_fracs[mf]))
            ax.yaxis.set_label_coords(1.1, 0.5) 
        if mf == 0:
            ax.set_title('t pulse = %s' % str(cases.pulse_durs[tp]))
        if mf == 2:
            ax.set_xlabel('time / s')
        if tp == 0:
            ax.set_ylabel('concentration / M')

In [ ]:
def plot_alkyls(xlim=None):
    fig, axes = plt.subplots(nmf, npd, figsize=(14, 14), sharey='col')
    for mf in range(nmf):
        for tp in range(npd):
            ax, r = axes[mf, tp], rs[mf, tp]
            x, y = r['tout'], total_of(r['Cout'], r['tokg']['a'])
            mi = np.argmax(y)
            tenth_y = y[mi]*0.1
            tenth_b = y[mi:] < tenth_y
            if np.any(tenth_b):
                tenth_i = mi + np.argmax(tenth_b)
            else:
                tenth_i = -1
            ax.plot(x[:tenth_i], y[:tenth_i])
            ax.ticklabel_format(axis='x', style='sci', scilimits=(-4, 4))
            ax.ticklabel_format(axis='y', style='sci', scilimits=(-4, 4))
            if tp == 2:
                ax.set_ylabel('mass frac = %s (%.3g M)' % (cases.mass_fracs[mf], r['ic0']))
                ax.yaxis.set_label_coords(1.1, 0.5) 
            if mf == 0:
                ax.set_title('t pulse = %s' % str(cases.pulse_durs[tp]))
            if mf == 2:
                ax.set_xlabel('time / s')
            if tp == 0:
                ax.set_ylabel('concentration / M')
            if xlim:
                ax.set_xlim(xlim)

In [ ]:
plot_alkyls(xlim=[0, 2e-3])

In [ ]:
plot_alkyls()

In [ ]:
def plot_tokens():
    fig, all_axes = plt.subplots(len(tokens)-1, npd, figsize=(16, 3*len(tokens)))
    for tki, (axes, tk) in enumerate(zip(all_axes, tokens[1:])):
        for pdi, ax in zip(range(npd), axes):
            for mfi in range(nmf):
                r = rs[mfi, pdi]
                c = viridis(mfi/(nmf-1.0))
                ax.plot(r['tout'], total_of(r['Cout'], r['tokg'][tk])/r['ic0'],
                        label='%s ppm' % cases.mass_fracs[mfi], c=c)
            ax.legend()
            if tki == 0:
                ax.set_title('Pulse duration: %s s' % cases.pulse_durs[pdi])
            if tki == len(tokens)-2:
                ax.set_xlabel('time / s')
            if pdi == 0:
                ax.set_ylabel(tk)
            
            ax.set_yscale('log')
            ax.set_ylim([1e-12, 1e2])

In [ ]:
plot_tokens()

In [ ]:
def plot_conc(sks=dict(H2O2=(1e-7, 1e-4, 'log'), O2=(1e-17, 1e-5, 'log'), N2O=(28.95e-3, 29.01e-3, 'linear'),
                       HO2=(1e-16, 1e-7, 'log'), OH=(1e-16, 1e-4, 'log'))):
    fig, all_axes = plt.subplots(len(sks)+1, npd, figsize=(16, 3*len(sks)))
    for pdi, ax in zip(range(npd), axes):
        for mfi in range(nmf):
            r = rs[mfi, pdi]
            lbl = 'mass frac: %s' % cases.mass_fracs[mfi]
            c = viridis(mfi/(nmf-1.0))
            for ski, (sk, (lb, ub, yscl)) in enumerate(sks.items()):
                ax = all_axes[ski, pdi]
                ax.plot(r['tout'], r['Cout'][sk], label=lbl, c=c)
                ax.set_ylim([lb, ub])
                ax.set_yscale(yscl)
                ax.set_xlim([1e-9, 300])
                ax.set_xscale('log')
                if pdi == 0 and mfi == 0:
                    ax.set_ylabel('[%s] / M' % sk)
            all_axes[-1, pdi].plot(r['tout'], average_length_by_weight(r['Cout'], r['tokg']), label=lbl, c=c)
        all_axes[-1, pdi].set_xlabel('time / s')
        all_axes[0, pdi].set_title('Pulse duration: %s s' % cases.pulse_durs[pdi])
        all_axes[-1, pdi].axhline(int(r['sk0'][1:].split('a')[0]), ls='--', lw=0.5, alpha=0.5, c='k')
        all_axes[-1, pdi].set_ylim([0, 500])
    all_axes[-1, 0].set_ylabel('Average length by weight')
    _ = all_axes[-1, 0].legend()

In [ ]:
plot_conc()